In [13]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [47]:
df = pd.read_csv('data/train2.csv', dtype={"district_id": "Int64", "city_id": "Int64", "id": "Int64", "is_business": "string", "region_id": "Int64", "price": "Int64", "created_at_first": "str",
"params": "str"}, engine="python", encoding="utf-8")

In [3]:
districts = pd.read_csv("data/districts.csv", dtype={'id': 'Int64'})

In [4]:
cities = pd.read_csv("data/cities.csv", dtype={'id': 'Int64'})

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [6]:
df.head()

,Unnamed: 0,id,created_at_first,category,is_business,district_id,city_id,region_id,params,price
0,0,325017,2018-10-06 12:20:48,Mieszkanie na sprzedaż,1,12867,210,14,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556
1,1,513427,2019-02-18 12:17:51,Mieszkanie na sprzedaż,1,<NA>,3614,15,price[currency]<=>PLN<br>m<=>95<br>rooms_num<=...,500000
2,2,824979,2019-03-21 17:31:09,Mieszkanie na sprzedaż,1,3434,204,10,price[currency]<=>PLN<br>m<=>58.9<br>rooms_num...,238000
3,3,400727,2018-11-28 23:50:45,Mieszkanie na sprzedaż,1,87,1,15,price[currency]<=>PLN<br>m<=>45<br>rooms_num<=...,209000
4,4,298324,2019-01-22 21:22:42,Dom na sprzedaż,0,<NA>,11879,7,price[currency]<=>PLN<br>terrain_area<=>3201<b...,729000


In [7]:
def merge_drop_rename(df, cities, districts):
    data = pd.merge(df, cities, left_on="city_id", right_on="id") #merging data with cities
    data = data.drop("id_y", axis=1) #dropping trash column
    districts = districts.rename(columns={"id": "district_id"})
    data = pd.merge(data, districts, on="district_id", how="left") #merging data (now data with cities) with districts
    data = data.drop(columns = {"Unnamed: 0", "id_x", "created_at_first", "region_id", "lon_x", "lat_x", "city_id", "district_id", "lon_y", "lat_y"}) #dropping trash columns
    data = data.rename(columns = {"name_pl_x": "city_name", "name_pl_y": "district_name"})
    return data

In [ ]:
def parse_values(row):
    pairs = row.split('<br>') #creating a pair of col_name and value
    values = {}
    for pair in pairs: #going through pairs of col_name and value
        if pair.strip() != '': 
            col, val = pair.split('<=>') #splitting a pair for col_name and value
            if val.strip() == '' or val.strip().lower() == 'nan':
                val = np.nan #filling with NaNs
            values[col] = val #putting values into columns
    return pd.Series(values)

In [16]:
def parse_extras(row):
    row = str(row) #for some reason it wasn't a string
    pairs = row.split('<->') #splitting extras
    values = {}
    for pair in pairs:
        if pair.strip() != '' and pair.strip() != 'nan' and pair.strip() != "0":
            col = pair
            val = True #assigning value
        else:
            col = "trash"
            val = np.nan
        values[col] = val #putting values into columns
    return pd.Series(values)

In [36]:
data = merge_drop_rename(df, cities, districts)
parsed_data = data['params'].apply(parse_values).apply(pd.Series) #extracting params

In [37]:
parsed_data.head()

,price[currency],m,rooms_num,market,building_type,floor_no,building_floors_num,building_material,windows_type,heating,build_year,construction_status,rent,rent[currency],building_ownership,free_from,media_types,security_types,equipment_types,extras_types,terrain_area,floors_num,garret_type,roof_type,roofing,location,recreational,fence_types,heating_types,access_types,vicinity_types,is_bungalow,use_types,type,dimensions,fence,deposit,deposit[currency],rent_to_students
0,PLN,72.14,4,secondary,NaN,floor_1,1,NaN,plastic,NaN,2018,NaN,NaN,PLN,NaN,NaN,NaN,NaN,NaN,balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PLN,95,4,secondary,ribbon,ground_floor,1,brick,plastic,gas,2014,ready_to_use,NaN,PLN,full_ownership,NaN,cable-television<->internet<->phone,closed_area<->roller_shutters,dishwasher<->fridge<->oven<->stove<->washing_m...,garage<->garden<->terrace<->two_storey<->usabl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PLN,58.9,3,secondary,block,floor_3,4,concrete_plate,wooden,urban,1986,to_renovation,430,PLN,full_ownership,NaN,cable-television<->internet,entryphone,fridge<->oven<->stove,balcony<->basement<->separate_kitchen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PLN,45,2,secondary,NaN,ground_floor,NaN,NaN,plastic,NaN,NaN,NaN,320,PLN,NaN,NaN,NaN,NaN,NaN,basement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PLN,195,6,secondary,detached,NaN,NaN,brick,plastic,NaN,2015,ready_to_use,NaN,PLN,NaN,NaN,cesspool<->electricity<->internet<->water,alarm<->anti_burglary_door<->closed_area<->ent...,NaN,attic<->garage<->pool,3201,one_floor,usable,diagonal,sheet,country,0,wire,coal,asphalt,forest,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
set(parsed_data["use_types"])

{'0', 'gastronomy<->hotel', nan, 'office'}

In [58]:
data = merge_drop_rename(df, cities, districts)
#data = data.loc[data['city_name'] == "Lublin"] #filtering data for a given city
parsed_data = data['params'].apply(parse_values).apply(pd.Series) #extracting params
#parsed_data = parsed_data.drop(["free_from", "building_ownership", "price[currency]", "rent[currency]", "use_types", "type", "dimensions", "fence", "deposit", "deposit[currency]", "rent_to_students"], axis=1)
columns_to_drop = ["construction_status", "free_from", "building_ownership", "price[currency]", "rent[currency]", "use_types", "type", "dimensions", "fence", "deposit", "deposit[currency]", "rent_to_students"]
existing_columns = [col for col in columns_to_drop if col in parsed_data.columns]
parsed_data = parsed_data.drop(columns=existing_columns)
to_parse = ["media_types", "security_types", "equipment_types", "extras_types"]
for column in to_parse:
    parsed_col = parsed_data[column].apply(parse_extras).apply(pd.Series) # for parsing extras_types 
    parsed_data = pd.concat([parsed_data, parsed_col], axis=1)
    parsed_data = parsed_data.drop(column, axis=1)
parsed_data = parsed_data.drop(["trash"], axis=1) #droping trash column from extras_type
#parsed_data = parsed_data.replace(np.nan, False)

In [51]:
parsed_data.head()

,m,rooms_num,market,building_type,floor_no,building_floors_num,building_material,windows_type,heating,build_year,rent,terrain_area,floors_num,garret_type,roof_type,roofing,location,recreational,fence_types,heating_types,access_types,vicinity_types,is_bungalow,cable-television,internet,phone,electricity,cable_television,gas,water,sewage,cesspool,water_purification,entryphone,anti_burglary_door,alarm,closed_area,monitoring,roller_shutters,fridge,furniture,oven,stove,washing_machine,dishwasher,tv,balcony,basement,garage,separate_kitchen,lift,garden,terrace,two_storey,usable_room,air_conditioning,attic,pool
19,57.1,3,secondary,block,floor_3,5,False,plastic,urban,1995,400,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,True,True,True,False,False,True,True,True,True,False,False,False,False,False,False,False,False
25,84.68,4,secondary,block,floor_3,4,False,False,urban,2001,500,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
250,52,2,secondary,block,floor_2,3,False,plastic,urban,2008,280,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,True,False,True,True,False,True,True,False,False,False,False,False,False,False,False,False,False
260,30.62,2,primary,apartment,floor_2,7,False,plastic,urban,2020,50,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,False
262,46,3,secondary,block,floor_4,4,concrete_plate,plastic,urban,1979,369.7,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [56]:
len(parsed_data)

12448

In [57]:
parsed_data.isnull().sum()/len(parsed_data)*100

m                       0.000000
rooms_num               0.056234
market                  0.000000
building_type          28.390103
floor_no               12.901671
building_floors_num    14.227185
building_material      55.117288
windows_type           14.484254
heating                35.475578
build_year             11.294987
rent                   56.675771
terrain_area           89.926093
floors_num             91.556877
garret_type            98.080013
roof_type              98.152314
roofing                94.521208
location               97.622108
recreational           97.589974
fence_types            95.163882
heating_types          95.212082
access_types           92.135283
vicinity_types         97.830977
is_bungalow            89.355720
cable-television       64.299486
internet               56.820373
phone                  67.432519
electricity            93.741967
cable_television       99.397494
gas                    93.444730
water                  95.236183
sewage    

In [9]:
data.head()

,category,is_business,params,price,city_name,district_name
0,Mieszkanie na sprzedaż,1,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556,Olsztyn,Dajtki
1,Mieszkanie na sprzedaż,1,price[currency]<=>PLN<br>m<=>95<br>rooms_num<=...,500000,Plewiska,NaN
2,Mieszkanie na sprzedaż,1,price[currency]<=>PLN<br>m<=>58.9<br>rooms_num...,238000,Białystok,Zielone Wzgórza
3,Mieszkanie na sprzedaż,1,price[currency]<=>PLN<br>m<=>45<br>rooms_num<=...,209000,Poznań,Łazarz
4,Dom na sprzedaż,0,price[currency]<=>PLN<br>terrain_area<=>3201<b...,729000,Wymysły,NaN


In [10]:
def parse_values(row):
    pairs = row.split('<br>') #creating a pair of col_name and value
    values = {}
    for pair in pairs: #going through pairs of col_name and value
        if pair.strip() != '': 
            col, val = pair.split('<=>') #splitting a pair for col_name and value
            if val.strip() == '' or val.strip().lower() == 'nan':
                val = np.nan #filling with NaNs
            values[col] = val #putting values into columns
    return pd.Series(values)

In [ ]:
def merge_drop_rename(df, cities, districts):
    data = pd.merge(df, cities, left_on="city_id", right_on="id") #merging data with cities
    data = data.drop("id_y", axis=1) #dropping trash column
    districts = districts.rename(columns={"id": "district_id"})
    data = pd.merge(data, districts, on="district_id", how="left") #merging data (now data with cities) with districts
    data = data.drop(columns = {"Unnamed: 0", "id_x", "created_at_first", "region_id", "lon_x", "lat_x", "city_id", "district_id", "lon_y", "lat_y"}) #dropping trash columns
    data = data.rename(columns = {"name_pl_x": "city_name", "name_pl_y": "district_name"})
    return data